# 💸 Projeto Final | Sistema de Controle Financeiro

# Etapa 1: Importação das Bibliotecas Necessárias e declaração de Lista para armazenar os registros

Permitindo salvar o arquivo no drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import json
from datetime import datetime
from enum import Enum
from typing import Optional, Union, Literal, Dict
from IPython.display import clear_output

In [ ]:
registros = []

class EnumTipoRegistro(Enum):
    RECEITA = 'RECEITA'
    DESPESAS = 'DESPESAS'
    INVESTIMENTOS = 'INVESTIMENTOS'

class EnumFiltragens(Enum):
    DATA = 'DATA'
    TIPO = 'TIPO'
    VALOR = 'VALOR'

# Etapa 2: Definição das Funções Básicas
Função para Criar Novos Registros

In [ ]:
def criar_registro(tipo: EnumTipoRegistro, valor: float, registros_atuais: list, taxa: float = None, data_registro: datetime = datetime.now()) -> None:
    """
        Cria um registro financeiro com base no tipo de transação e valor fornecidos.

        Args:
            tipo (EnumTipoRegistro): O tipo de registro, que pode ser RECEITA, DESPESAS ou INVESTIMENTO.
            valor (float): O valor monetário do registro. Para despesas, o valor será convertido em negativo.
            taxa (float): A taxa de rendimento para investimentos.
            registros_atuais (list): A lista de registros que será atualizada com o novo registro.
            data_registro (datetime, opcional): A data do registro. Se não for fornecida, a data atual será usada.

        Returns:
            None
    """
    montante: Optional[float] = None

    if tipo == EnumTipoRegistro.DESPESAS:
        valor = -abs(valor)
    elif tipo == EnumTipoRegistro.INVESTIMENTOS:
        montante = calcular_rendimento(valor, data_registro, taxa)

    registro = {
        'DATA': data_registro.strftime('%Y-%m-%d'),
        'DIA': data_registro.day,
        'MES': data_registro.month,
        'ANO': data_registro.year,
        'TIPO': tipo.value,
        'VALOR': valor,
        'TAXA': taxa or 'X',
        'MONTANTE': montante or 'X'
    }

    registros_atuais.append(registro)

def calcular_rendimento(capital: float, data_investimento: datetime, taxa: float = 0.01) -> float:
    """
        Calcula o montante de um investimento com base na capital inicial, data de investimento e taxa de rendimento.

        Args:
            capital (float): O valor inicial do investimento.
            data_investimento (datetime): A data em que o investimento foi realizado.
            taxa (float, opcional): A taxa de rendimento diária. O padrão é 0.01 (1%).

        Returns:
            float: O montante acumulado do investimento até a data atual.
    """
    dias = (datetime.now() - data_investimento).days
    montante = round(capital * (1 + taxa) ** dias, 2)
    return montante

Função para Ler Registros

In [ ]:


def ler_registros(filtro: EnumFiltragens, valor: Union[str, float, EnumTipoRegistro], registros: list) -> list:
    """
        Filtra uma lista de registros financeiros com base no critério especificado.

        Args:
            filtro (EnumFiltragens): O critério de filtragem (DATA, TIPO, VALOR).
            valor (Union[str, float, EnumTipoRegistro]): O valor a ser comparado para a filtragem.
            registros (list): A lista de registros a ser filtrada.

        Returns:
            list: Uma lista de registros que correspondem ao critério de filtragem.
    """

    def filtrar_por_data():
        return [r for r in registros if r['data'] == valor]

    def filtrar_por_tipo():
        tipo_valor = valor.value if isinstance(valor, EnumTipoRegistro) else valor
        return [r for r in registros if r['tipo'] == tipo_valor]

    def filtrar_por_valor():
        return [r for r in registros if r['valor'] == valor]

    switch = {
        EnumFiltragens.DATA: filtrar_por_data,
        EnumFiltragens.TIPO: filtrar_por_tipo,
        EnumFiltragens.VALOR: filtrar_por_valor
    }

    return switch.get(filtro, lambda: registros)()


Função para Atualizar Registros

In [ ]:
def atualizar_registro(indice: int, registros: list, tipo: Optional[EnumTipoRegistro] = None, valor: Optional[float] = None, taxa: Optional[float] = None) -> None:
  """
    Atualiza um registro na lista de registros financeiros.

    Args:
        indice (int): O índice do registro a ser atualizado.
        registros (list): A lista de registros financeiros.
        tipo (Optional[EnumTipoRegistro]): O novo tipo de transação (opcional).
        valor (Optional[float]): O novo valor da transação (opcional).
        taxa (Optional[float]): A nova taxa de rendimento (opcional).
  """
  if 0 <= indice < len(registros):
    registro = registros[indice]

    if tipo is not None:
      if tipo == EnumTipoRegistro.INVESTIMENTOS:
        registro['MONTANTE'] = calcular_rendimento(valor, datetime.now(), taxa)
      registro[EnumFiltragens.TIPO.value] = tipo.value
    if valor is not None:
      registro[EnumFiltragens.VALOR.value] = valor if tipo != EnumTipoRegistro.DESPESAS else -abs(valor)

    registro[EnumFiltragens.DATA.value] = datetime.now().strftime('%Y-%m-%d')
  else:
    raise IndexError(f"Índice fora do intervalo: {indice}")

Função para Deletar Registros

In [ ]:
def deletar_registro(indice: int, registros: list) -> None:
    """
        Deleta um registro da lista de registros financeiros.

        Args:
            indice (int): O índice do registro a ser deletado.
            registros (list): A lista de registros financeiros.
    """
    if 0 <= indice < len(registros):
        del registros[indice]
    else:
        raise IndexError(f"Índice fora do intervalo: {indice}")

# Etapa 3: Funções Adicionais

Função para Atualizar Rendimento

In [ ]:
def atualiza_rendimento(registros: list) -> None:
    """
      Atualiza os valores de rendimento para todos os registros de investimento.

      Args:
        registros (list): A lista de registros financeiros.
    """
    for registro in registros:
        if registro['TIPO'] == EnumTipoRegistro.INVESTIMENTOS.value:
            capital = registro['VALOR']
            data_investimento = datetime.strptime(registro['DATA'], '%Y-%m-%d')
            taxa = registro['TAXA']
            registro['MONTANTE'] = round(calcular_rendimento(capital, data_investimento, taxa), 2)

Função para Exportar Relatório

In [ ]:
def exportar_relatorio(formato: Literal['csv', 'json']  = 'csv') -> None:
    """
        Exporta o relatório no formato especificado.

        Args:
            formato (Literal['csv', 'json']): O formato de exportação, 'csv' ou 'json'. O padrão é 'csv'.
    """
    _exportar_csv() if formato == 'csv' else _exportar_json() if formato == 'json' else None

def _exportar_csv() -> None:
    """
        Exporta os registros para um arquivo CSV.
    """
    with open('relatorio.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=registros[0].keys())
        writer.writeheader()
        writer.writerows(registros)

def _exportar_json() -> None:
    """
        Exporta os registros para um arquivo JSON.
    """
    with open('relatorio.json', 'w') as file:
        json.dump(registros, file, indent=4)

Função de Agrupamento

In [ ]:
def agrupar_registros(registros: list, criterio: str) -> dict:
    """
        Agrupa os registros por um critério especificado e calcula o total de valores para cada grupo.

        Args:
            registros (list): A lista de registros financeiros.
            criterio (str): O critério de agrupamento ('TIPO', 'DIA', 'MES', 'ANO').

        Returns:
            dict: Um dicionário com o total de valores para cada grupo.
    """
    agrupamento = {}
    for registro in registros:
        chave = registro.get(criterio)
        if chave is not None:
            if chave not in agrupamento:
                agrupamento[chave] = 0
            agrupamento[chave] += registro['VALOR']
    return agrupamento

Função para printar o relatório no terminal

In [ ]:
def printar_relatorio() -> None:
    """
    Imprime o relatório de registros financeiros de forma tabular.

    O relatório é lido de um arquivo CSV e formatado para exibição em uma tabela.

    Args:
        Nenhum argumento é necessário.
    """
    idx = 0
    with open("relatorio.csv", mode='r') as arquivo_csv:
        conteudo = csv.reader(arquivo_csv, delimiter=';')
        for linha in conteudo:
            linha = linha[0].replace(',', ' | ')
            if idx == 0:
                print(f"{'Indice':<10} | {linha:<30}")
                print(f"{'-'*80}")
            else:
                print(f"{idx:<10} | {linha:<30}")
            idx += 1

In [ ]:
printar_relatorio()

# Etapa 4: Testando as Funções

In [ ]:
registros = []

In [ ]:
try:
    tipo = input("Digite o tipo de registro (RECEITA, DESPESAS, INVESTIMENTOS): ").upper()
    if tipo not in EnumTipoRegistro.__members__:
        raise ValueError("Tipo de registro inválido.")
    tipo_registro = EnumTipoRegistro[tipo]

    valor = float(input("Digite o valor: "))
    if valor <= 0:
        raise ValueError("O valor deve ser positivo.")

    if tipo_registro == EnumTipoRegistro.INVESTIMENTOS:
        taxa = float(input("Digite a taxa de rendimento: "))
        if taxa <= 0:
            raise ValueError("A taxa de rendimento deve ser positiva.")
    else:
        taxa = 'X'

    data = input('Digite a data no formato dia/mês/ano ou deixe em branco para usar data atual')
    if data.strip() != '':
        try:
            data_datetime = datetime.strptime(data, "%d/%m/%Y")
            criar_registro(tipo_registro, valor, registros, taxa, data_datetime)
        except ValueError:
            criar_registro(tipo_registro, valor, registros, taxa)
    else:
        criar_registro(tipo_registro, valor, registros, taxa)




except ValueError as ve:
    print(f"Erro de valor: {ve}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

print(registros)

In [ ]:
exportar_relatorio('csv')
exportar_relatorio('json')

Lendo Registros

In [ ]:
registrosFiltrados = []

try:
    tipo_filtro = input("Deseja filtrar por Valor, Tipo ou Data? ").strip().upper()

    if tipo_filtro == 'VALOR':
        valor_filtro = float(input("Digite o valor para filtrar os registros: "))
        for registro in registros:
            if registro["VALOR"] == valor_filtro:
                registrosFiltrados.append(registro)

    elif tipo_filtro == 'TIPO':
        tipo_registro_filtro = input("Digite o tipo de registro para filtrar (RECEITA, DESPESAS, INVESTIMENTOS): ").upper()
        if tipo_registro_filtro not in EnumTipoRegistro.__members__:
            raise ValueError("Tipo de registro inválido.")
        for registro in registros:
            if registro["TIPO"] == tipo_registro_filtro:
                registrosFiltrados.append(registro)

    elif tipo_filtro == 'DATA':
        data_filtro_str = input("Digite a data para filtrar os registros (dia/mês/ano): ")
        data_filtro = datetime.strptime(data_filtro_str, "%d/%m/%Y").strftime('%Y-%m-%d')
        for registro in registros:
            if registro["DATA"] == data_filtro:
                registrosFiltrados.append(registro)
    else:
        raise ValueError("Opção de filtro inválida.")

except ValueError as ve:
    print(f"Erro de valor: {ve}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

print(registrosFiltrados)

Atualizando um Registro e deletando

In [ ]:
try:
    acao = input("Você deseja Atualizar ou Deletar um registro? ").strip().upper()
    if acao.upper() not in ['ATUALIZAR', 'DELETAR']:
        raise ValueError("Ação inválida.")
    else:
      printar_relatorio()
      if acao == 'ATUALIZAR':
          index = int(input("Digite o índice do registro que deseja atualizar: ")) - 1
          if index < 0 or index >= len(registros):
              raise IndexError("Índice inválido.")

          novo_tipo = input("Digite o novo tipo de registro (RECEITA, DESPESAS, INVESTIMENTOS): ").upper()
          if novo_tipo not in EnumTipoRegistro.__members__:
              raise ValueError("Tipo de registro inválido.")
          tipo_registro = EnumTipoRegistro[novo_tipo]

          novo_valor = float(input("Digite o novo valor: "))
          if novo_valor <= 0:
              raise ValueError("O valor deve ser positivo.")

          if novo_tipo.upper() == 'INVESTIMENTOS':
              nova_taxa = float(input("Digite a nova taxa de rendimento: "))
              if nova_taxa <= 0:
                  raise ValueError("A taxa de rendimento deve ser positiva.")

          atualizar_registro(index, registros, EnumTipoRegistro[novo_tipo], novo_valor)

      elif acao == 'DELETAR':
          index = int(input("Digite o índice do registro que deseja deletar: ")) - 1
          if index < 0 or index >= len(registros):
              raise IndexError("Índice inválido.")

          del registros[index]

      else:
          raise ValueError("Ação inválida.")

except ValueError as ve:
    print(f"Erro de valor: {ve}")
except IndexError as ie:
    print(f"Erro de índice: {ie}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

print(registros)

In [ ]:
exportar_relatorio('csv')
exportar_relatorio('json')

Agrupando por Tipo

In [ ]:
try:
    tipo_agrupamento = input('Você deseja agrupar os registros por Valor, Tipo ou Data? ').strip().upper()

    if tipo_agrupamento == 'VALOR':
        agrupamento = agrupar_registros(registros, 'VALOR')

    elif tipo_agrupamento == 'TIPO':
        agrupamento = agrupar_registros(registros, 'TIPO')

    elif tipo_agrupamento == 'DATA':
        agrupamento = agrupar_registros(registros, 'DATA')
    else:
        raise ValueError("Opção de filtro inválida.")

except ValueError as ve:
    print(f"Erro de valor: {ve}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

print(agrupamento)


Atualizações de Montantes

In [ ]:
atualizar_rendimento = input('Gostaria de atualizar os rendimentos? (S/N)').strip().upper()

if atualizar_rendimento == 'S':
  print('atualizou')
  atualiza_rendimento(registros)


for registro in registros:
    print(registro)

In [ ]:
exportar_relatorio('csv')
exportar_relatorio('json')

In [ ]:
while True:
    clear_output(wait=True)
    operacao = input("""
        Qual operação voce deseja executar?\n
        1 - Criar um novo registro \n
        2 - Atualizar um registro \n
        3 - Excluir um registro \n
        4 - Visualizar relatório completo \n
        5 - Exportar relatório \n
        6 - Agrupar registros \n
        7 - Atualizar rendimentos \n
        0 - Sair \n
    """).strip()
    if operacao.isdigit():
        operacao = int(operacao)
        if operacao == 0:
            print("Até mais!")
            break
        elif operacao == 1:
          try:
              tipo = input("Digite o tipo de registro (RECEITA, DESPESAS, INVESTIMENTOS): ").upper()
              if tipo not in EnumTipoRegistro.__members__:
                  raise ValueError("Tipo de registro inválido.")
              tipo_registro = EnumTipoRegistro[tipo]

              valor = float(input("Digite o valor: "))
              if valor <= 0:
                  raise ValueError("O valor deve ser positivo.")

              if tipo_registro == EnumTipoRegistro.INVESTIMENTOS:
                  taxa = float(input("Digite a taxa de rendimento: "))
                  if taxa <= 0:
                      raise ValueError("A taxa de rendimento deve ser positiva.")
              else:
                  taxa = 'X'

              data = input('Digite a data no formato dia/mês/ano ou deixe em branco para usar data atual')
              if data.strip() != '':
                  try:
                      data_datetime = datetime.strptime(data, "%d/%m/%Y")
                      criar_registro(tipo_registro, valor, registros, taxa, data_datetime)
                  except ValueError:
                      criar_registro(tipo_registro, valor, registros, taxa)
              else:
                  criar_registro(tipo_registro, valor, registros, taxa)
          except ValueError as ve:
              print(f"Erro de valor: {ve}")
          except Exception as e:
              print(f"Ocorreu um erro: {e}")

        elif operacao == 2:
          printar_relatorio()
          index = int(input("Digite o índice do registro que deseja atualizar: ")) - 1
          if index < 0 or index >= len(registros):
              raise IndexError("Índice inválido.")

          novo_tipo = input("Digite o novo tipo de registro (RECEITA, DESPESAS, INVESTIMENTOS): ").upper()
          if novo_tipo not in EnumTipoRegistro.__members__:
              raise ValueError("Tipo de registro inválido.")
          tipo_registro = EnumTipoRegistro[novo_tipo]

          novo_valor = float(input("Digite o novo valor: "))
          if novo_valor <= 0:
              raise ValueError("O valor deve ser positivo.")

          if novo_tipo.upper() == 'INVESTIMENTOS':
              nova_taxa = float(input("Digite a nova taxa de rendimento: "))
              if nova_taxa <= 0:
                  raise ValueError("A taxa de rendimento deve ser positiva.")

          atualizar_registro(index, registros, EnumTipoRegistro[novo_tipo], novo_valor)
        elif operacao == 3:
          printar_relatorio()
          index = int(input("Digite o índice do registro que deseja deletar: ")) - 1
          if index < 0 or index >= len(registros):
              raise IndexError("Índice inválido.")

          del registros[index]

        elif operacao == 4:
          printar_relatorio()
        elif operacao == 5:
            exportar_relatorio('csv')
            exportar_relatorio('json')
        elif operacao == 6:
          try:
              tipo_agrupamento = input('Você deseja agrupar os registros por Valor, Tipo ou Data? ').strip().upper()

              if tipo_agrupamento == 'VALOR':
                  agrupamento = agrupar_registros(registros, 'VALOR')

              elif tipo_agrupamento == 'TIPO':
                  agrupamento = agrupar_registros(registros, 'TIPO')

              elif tipo_agrupamento == 'DATA':
                  agrupamento = agrupar_registros(registros, 'DATA')
              else:
                  raise ValueError("Opção de filtro inválida.")

          except ValueError as ve:
              print(f"Erro de valor: {ve}")
          except Exception as e:
              print(f"Ocorreu um erro: {e}")

          print(agrupamento)
        elif operacao == 7:
          atualizar_rendimento = input('Gostaria de atualizar os rendimentos? (S/N)').strip().upper()

          if atualizar_rendimento == 'S':
            print('atualizou')
            atualiza_rendimento(registros)


          for registro in registros:
              print(registro)
        else:
            print("Opção inválida")
            continue
    else:
        print("Opção inválida")
        continue


# Extra: Transformando o nosso arquivo csv em google spreadsheet e salvando-o no google drive

Importanto as bibliotecas


In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload


Realizando a autenticação para salvar o arquivo no drive

In [ ]:
auth.authenticate_user()

drive_service = build('drive', 'v3')

Criando o arquivo .spreadsheet e salvando o mesmo no drive

In [ ]:
file_path = 'relatorio.csv'

file_metadata = {
    'name': 'relatorio_financeiro_ada',
    'mimeType': 'application/vnd.google-apps.spreadsheet'
}

media = MediaFileUpload(file_path, mimetype='text/csv', resumable=True)
created_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

print(f'Arquivo criado com ID: {created_file.get("id")}')
